Possible Paper Titles

1) Improving LLM conversational recommendations: How Empathy Leads to Information Gain

2) SwIG: Sampling with Information Gain to improve conversational recommendations

## Connecting to Drive and Loading Requirements


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
drive_dir = '/content/drive/MyDrive/Clarifying_Questions_GPT_Research'

In [3]:
!cd {drive_dir} && pip install -r requirements_2.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.4 MB/s eta 0:00:00


In [4]:
import openai

import pickle

import json

import threading

import numpy as np

import random

import re

from enum import Enum

from collections import Counter

import math

from typing import List

import copy

import signal

import time

import sys

import os

from tqdm import tqdm

import pprint

import urllib3

In [5]:
with open(drive_dir+"/openai_key.key", "r") as f:
    openai.api_key = f.read()

## Basic Functions and Decorators

In [6]:
# given a string that has a list, creates a python list with the input's contents
def str2lst(l):

  # if it is a pyhton list as a str
  if '[' in l and ']' in l:
    # removing double spaces and newlines
    l = re.sub("\s+", " ", l)
    # removing anything that isn't the list
    m = re.search('\[(.|\s)+\]', l)
    l = l[m.start():m.end()]
    # converting string to list
    l = l.strip('][')
    l = re.split("""['"], """, l)

    # spaghetti code to dealing with this weird bug I saw before
    if len(l) == 1:
      l = l[0]
      l = re.sub("-", "", l)
      l = l.split('\n')

  # else if it is bullet points tha are...
  #numbered
  elif re.search('\d\.', l):
    # converting string to list
    l = re.findall('\d\.\s(.+)', l)

  #not numbered
  else:
    # converting string to list
    l = re.split('-', l)[1:]

  # removing extra quotation marks
  for i in range(len(l)): l[i] = re.sub("""['"]""", "", l[i])
  # removing trailing and starting spaces
  for i in range(len(l)): l[i] = l[i].strip(' ')

  return l

# function to turn a python list into a numbered list as a string
def list2numbered(lst):
  num_lst = []
  for idx in range(len(lst)): num_lst.append(f'{idx+1}. {lst[idx]}\n')
  return ''.join(num_lst)

# decorator that saves the output of func into a specific index of a lst (for multi threading)
def saveInLst(lst, idx):

  def inner(func):

    def wrap(*args, **kwargs):
      lst[idx] = func(*args, **kwargs)
      return

    return wrap

  return inner

# given a function, a list of input lists, creates a thread for each input list, running the function with different inouts in parallel. Returns a list of outputs
def multiThread(func, input_lst):
  thread_n = len(input_lst)
  threads = []

  results = [None for i in range(thread_n)]

  n = 80
  idxs = [i for i in range(thread_n )]
  batch_idxs = [idxs[i * n:(i + 1) * n] for i in range((len(idxs) + n - 1) // n )]
  # print(batch_idxs)
  for b in batch_idxs:
    # print(b)
    for i in b:
      time.sleep(2) # my attempt to stop hitting the rate limit
      tr = threading.Thread(target=saveInLst(results, i)(func), args=(*input_lst[i],))
      # tr = threading.Thread(target=func, args=(*input_lst[i],))
      tr.start()
      threads.append(tr)

    for tr in threads:
      tr.join()

  return results

## Functions for Text Generation

In [7]:
# generates text using gpt-3.5 given a conversation context, returns None if its not able to generate text after a certain number of tries
def generateText(context, temp = 0.7, tries = 3, max_time = 120):
  assert isinstance(context, list), f'input is not a context list, got {type(context)}'

  resp = None

  while tries != 0:
    try:
      resp = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=context,
        temperature=temp)['choices'][0]['message']['content']

      tries = 0
    except:
      tries -= 1

  return resp


# def generateMText(context, thread_n, temp = 0.7, tries = 3):
#   threads = []

#   results = [None for i in range(thread_n)]

#   seed = 'Generate a profile of a user who is looking to travel. The profile should be comprised of a few bullet points that describe them'

#   for i in range(thread_n):
#     tr = threading.Thread(target=saveInLst(results, i)(generateText), args=(context, temp, tries,))
#     tr.start()
#     threads.append(tr)

#   for tr in threads:
#     tr.join()

#   return results

# appends a new entry to a conversation context
def appendContext(text, context, role = 'user'):
  assert role == 'user' or role == 'assistant', f'unexpected role, got: {role}'

  if not context:
    context.append({'role': role, 'content': text})
    return

  if role == 'user':
    assert context[-1]['role'] == 'assistant', f'incompatible adjecent role, got user'
  else:
    assert context[-1]['role'] == 'user', f'incompatible adjecent role, got assistant'

  context.append({'role': role, 'content': text})

  return

# Prompts

In [8]:
def stichPrompt(prompt, var_lst):
  var_places = len(re.findall('{\S*}', prompt))
  assert var_places == len(var_lst), f'prompt has to have the same number of places for variables as variables in var_lst. Prompt has {var_places} var places and len(var_lst) = {len(var_lst)}. Here is the prompt:\n{prompt}'
  for var_idx in range(var_places):
    m = re.search('{\S*}', prompt)
    start = prompt[:m.start()]
    end = prompt[m.end():]
    prompt = f'{start}{var_lst[var_idx]}{end}'

  return prompt

# CREATE CODE THAT SERVES AS DICT OF ALL PROPMTS (WHERE THE VALUES AZRE FUNCTIONS)
stichPrompt("Here is an example prompt, with a var here: {var} and here: {}", [1,2])

'Here is an example prompt, with a var here: 1 and here: 2'

In [9]:
prompts = {'startConv': ["""You are taking an English test.\nHere is a sentence: "{i_want_sen}, but I do not know [insert word]. Help me by asking me a question at a time."\nReturn the sentence but with the correct word filled in the empty slot:"""],

           'genRealProfile': ["Generate a profile of a user. The profile should be a list of keywords about their preferences regarding {domain}."],

           'continueConvBool': ["Is the following text a question or a suggestion?/n/n{text}/n/nAnswer [Question, Suggestion]:"],

           'genUProfiles': ['Generate {number} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding {domain}.'],

           'extractLastPreference': ['Here is a conversation.\nASSISTANT:\n{question}\nUSER:\n{answer}\nTASK: Summarize what we know about the user in a single sentence.\nSUMMARY:'],

           'genYProfiles': ['Here is a list of user preferences:{bullet_points}',
                            'Here is a sentence describing the preferences of a user:{sentence}\n',
                            'Create {number} different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:',
                            'Create different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'],

           'answerQ': ['You are a character. Here is what you know about your character: {profile}\nYou are speaking to an assistant and you speak in brief sentences.\nAnswer the assistant in character.\nAssistant: {question}\nYou:'],

           'pickBranch': ["""You are the following character: {profile}\nYou are asked the following question: {question}\n\nFrom the options below, which is most likely your answer to the question?\n{bullet_points}\nRemember, you have to pick one from the list or answer "None of the answers in the list". Only return the most likely answer, followed by the number that is the place of the answer in the list.\nAnswer, Number:""",
                          "You are the following character: {profile}\nYou are asked the following question: {question}\nIs it likely that you would answer: {answer}?\n\n[Yes, No]:"],

          'makeBranches': ['Here is a question:\n"{question}"\nReturn the smallest numbered list of different possible answers to this question. Make sure that this list, althought small, covers how any user could answer this question.'],

           'makeNodes': ['Generate a python list of {node_n} questions you could possibly ask me at this point of the conversation.',
                         'Generate a python list of questions you could possibly ask me at this point of the conversation.',
                         ' Inlcude the last question you just asked in the python list.',
                         "Let's say I said the following: {answer}\n {prompt}"],


           }

# Functions for Profiles

In [10]:
# Create U Profiles
def genUProfiles(prof_num = 50, domain = 'movies'):
  prompt = stichPrompt(prompts['genUProfiles'][0], [prof_num, domain])
  #prompt = f'Generate {prof_num} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding traveling.'
  return [Profile(txt, ProfileType.U) for txt in str2lst(generateText([{'role': 'user', 'content': prompt}], tries = 2))]

# Extract the last preference the user revealed in the chat context
def extractLastPreference(chat_hist):
  chat_hist = chat_hist[-2:]
  assert chat_hist[0]['role'] == 'assistant', 'chat history should have the assistant as the second to last message'
  assert chat_hist[1]['role'] == 'user', 'chat history should have the user as the last message'

  prompt = stichPrompt(prompts['extractLastPreference'][0], [chat_hist[0]['content'], chat_hist[1]['content']])
  # prompt = f'Here is a conversation.\nASSISTANT:\n{chat_hist[0]['content']}\nUSER:\n{chat_hist[1]['content']}\nTASK: Summarize what we know about the user in a single sentence.\nSUMMARY:'

  return generateText([{'role': 'user', 'content': prompt}], tries = 2)

# Create Y Profiles
def genYProfiles(preference_lst = [], prof_num = None):
  p_n = len(preference_lst)
  assert p_n != 0, 'need at least one preference in preference_lst'

  prompt = preference_lst[0]

  # if preference_lst has more than oone preferece, create a single sentence representing all of them
  if p_n > 1:
    prompt = stichPrompt(prompts['genYProfiles'][0], [list2numbered(preference_lst)])
    # prompt = f'Here is a list of user preferences:{list2numbered(preference_lst)}'
    prompt = generateText([{'role': 'user', 'content': prompt+'Create a single sentence of key words that describes all of these preferences:'}], tries = 2)

  prompt = stichPrompt(prompts['genYProfiles'][1], [prompt])
  # prompt = f'Here is a sentence describing the preferences of a user:{prompt}\n'

  # if profile number is specified, generate this specific number, otherwise let LLM decide
  if prof_num:
    prompt = stichPrompt(prompts['genYProfiles'][2], [prof_num])
    # prompt += f'Create {prof_num} different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'
  else:
    prompt += prompts['genYProfiles'][3]
    # prompt += 'Create different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'

  return [Profile(txt, ProfileType.Y) for txt in str2lst(generateText([{'role': 'user', 'content': prompt}], tries = 2))]

# Create a real profile
def genRealProfile(domain = 'movies'):
  prompt = stichPrompt(prompts['genRealProfile'][0], [domain])
  #prompt = f'Generate {prof_num} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding traveling.'
  return Profile(generateText([{'role': 'user', 'content': prompt}], tries = 2), ProfileType.R)

# Tree Structure

In [11]:
# Enum of Profile types
class ProfileType(Enum):
  U = 0
  Y = 1
  R = 2

# Class representing a user profile
class Profile:
  def __init__(self, text: str, p_type: ProfileType):
    self.text = text
    self.p_type = p_type
    return

  # # makes this Proifle into a UProfile
  # def makeUProfile(self):
  #   self.p_type = ProfileType.U
  #   return

  # # makes this Proifle into a YProfile
  # def makeYProfile(self, context):
  #   self.p_type = ProfileType.Y
  #   return

  # answers question q as the user the profile describes, returns string
  @staticmethod
  def answerQ(self, q):
    prompt = stichPrompt(prompts['answerQ'][0], [self.text, q])
    # prompt = f'You are a character. Here is what you know about your character: {self.text}\nYou are speaking to an assistant and you speak in brief sentences.\nAnswer the assistant in character.\nAssistant: {q}\nYou:'
    return generateText([{'role': 'user', 'content': prompt}])

  # picks branch that best fits profile, returns int that is index of picked branch in the branches list or None
  @staticmethod
  def pickBranch(self, node):
    prompt = stichPrompt(prompts['pickBranch'][0], [self.text, node.question, list2numbered(node.branches)])
    # prompt = f"""You are the following character: {self.text}\nYou are asked the following question: {node.question}\n\nFrom the options below, which is most likely your answer to the question?\n{list2numbered(node.branches)}\nRemember, you have to pick one from the list or answer "None of the answers in the list". Only return the most likely answer, followed by the number that is the place of the answer in the list.\nAnswer, Number:"""
    txt = generateText([{'role': 'user', 'content': prompt}], temp=1.0)

    # trying to find a number in the answer generated
    n_m = re.search('\d', txt)

    if n_m:
      # if the number chosen is not an index to the branches list, return None
      branch_idx = int(txt[n_m.start():n_m.end()]) - 1
      if branch_idx >= len(node.branches):
        return

      # if given an appropriate answer, make sure the answer shouldn't be neither
      prompt = stichPrompt(prompts['pickBranch'][1], [self.text, node.question, node.branches[branch_idx]])
      # prompt = f"You are the following character: {self.text}\nYou are asked the following question: {node.question}\nIs it likely that you would answer: {txt[t_m.start():t_m.end()]}?\n\n[Yes, No]:"

      if 'yes' in generateText([{'role': 'user', 'content': prompt}]).lower():
        return branch_idx

    # if no number was given, or if the number was not an appropriate answer, return None
    return

# Class representing a node in a layer of the "decision tree"
class Node:
  def __init__(self, q : str):
    # question that represents this node
    self.question = q
    # possible branches/answers to this node/question
    self.branches = None
    self.tst_b = None
    # map of profile index (from given profile_lst input in splitProfiles func) to branch index
    self.profile_branch_map = None
    # entropy and info gain for this Node
    self.entropy = None
    self.info_gains = None

    return

  # generates branches for this node
  @staticmethod
  def makeBranches(self):
    assert self.question != None, "self.question cannot be None"

    prompt = stichPrompt(prompts['makeBranches'][0], [self.question])
    # prompt = f'Here is a question:\n"{self.question}"\nReturn a concise list of different possible answers to the question.'
    self.tst_b = generateText([{'role': 'user', 'content': prompt}], tries = 2, temp = 1.0)
    self.branches = str2lst(self.tst_b)
    return

  # splits profiles into the different branches, then creates a mapping from profile index to branch index
  @staticmethod
  def splitProfiles(self, profile_lst : List[Profile]):
    assert self.branches != None, "self.branches cannot be None"
    assert len(profile_lst) != 0, "self.profile_lst cannot be empty"

    # each profile picks branch that is best fit, returns list of int
    # saving what profile picked what branch in self.profile_branch_map
    self.profile_branch_map = multiThread(Profile.pickBranch, [[p, self] for p in profile_lst])

    # editing self.branches and self.profile_branch_map if a profile picked no Branches
    has_none = False
    for idx in range(len(self.profile_branch_map)):
      if self.profile_branch_map[idx] == None:
        self.profile_branch_map[idx] = len(self.branches)
        has_none = True

    if has_none:
      self.branches.append("NONE OF THE ABOVE")

    return

  # calculating info gain
  def clalcInfoGain(self, profile_lst : List[Profile]):
    # number of total profiles
    total_profiles_n = len(profile_lst)
    # creating a set of classes
    classes = set()
    for p in profile_lst:
      classes.add(p.p_type)
    # map from branch index to count of profiles that picked this branch
    branch_count = Counter(self.profile_branch_map)
    # map from branch index to possible node entropy
    branch_entropy = []

    # if there is only one class, treat every profile as its own class
    if len(classes) == 1:
      # calculating this node's entropy
      self.entropy = -1*math.log2(1/total_profiles_n)

      # calculating the entropy of each branch
      for b_idx in range(len(self.branches)):
        if branch_count[b_idx] != 0: branch_entropy.append(-1*math.log2(1/branch_count[b_idx]))
        else: branch_entropy.append(0)

    # otherwise
    else:
      # calculating this node's entropy
      self.entropy = 0
      for c in Counter([p.p_type.value for p in profile_lst]).values(): self.entropy -= (c/total_profiles_n)*math.log2(c/total_profiles_n)

      # calculating the entropy of each branch
      assert 1==0

    # calculating information gain
    branch_entropy_weighted_sum = 0
    for b_idx in range(len(self.branches)):
      branch_entropy_weighted_sum += (branch_count[b_idx]/total_profiles_n) * branch_entropy[b_idx]

    self.info_gains = self.entropy - branch_entropy_weighted_sum
    return


# Class representing a layer of the "decision tree"
class Layer:
  def __init__(self, context):
    # current context
    self.context = context
    # list of nodes, each representing a possible question for this layer of the conversation
    self.nodes = None

    # list of Profile
    self.profiles = []

    # index of the node with the highest information gain
    self.best_node_idx = None

    # stores next layer
    self.next = None
    # stores previous layer
    self.prev = None

    return

  # creating U Profiles
  def makeUProfiles(self):
    return

  # creating nodes, hence possible questions for this layer
  def makeNodes(self, node_n = None):
    # making a deep copy of the context so it is unchanged
    context = copy.deepcopy(self.context)

    # making the base prompt
    if node_n:
      prompt = stichPrompt(prompts['makeNodes'][0], [node_n])
      # prompt = f'Generate a python list of {node_n} questions you could possibly ask me at this point of the conversation.'
    else:
      prompt = prompts['makeNodes'][1]
      # prompt = 'Generate a python list of questions you could possibly ask me at this point of the conversation.'

    if context[-1]['role'] == 'assistant':
      prompt += prompts['makeNodes'][2]
      # prompt += ' Inlcude the last question you just asked in the python list.'
    else:
      prompt = stichPrompt(prompts['makeNodes'][3], [context.pop()['content'], prompt])
      # prompt = f"Let's say I said the following: {context.pop()['content']}\n {prompt}"


    # appending the prompt to context
    appendContext(prompt, context, 'user')

    # generating the questions and storing them as a python list, then initializing nodes
    self.nodes = [Node(q) for q in str2lst(generateText(context, tries = 2))]
    return

  # creates branches for each Node in self.nodes
  def makeBranches(self):
    assert self.nodes != None, "self.nodes cannot be None"
    multiThread(Node.makeBranches, [[n] for n in self.nodes])
    return

  # create a mapping from profiles to branches for each node
  def splitProfiles(self):
    assert self.nodes != None, "self.nodes cannot be None"
    assert len(self.profiles) != 0, "self.profiles cannot be empty"
    multiThread(Node.splitProfiles, [[n, self.profiles] for n in self.nodes])
    return
    # for n in self.nodes:
    #   n.splitProfiles(self.profiles)
    # return

  # calculates info gain for every node and saves index of best node
  def calcInfoGain(self):
    for n in self.nodes: n.clalcInfoGain(self.profiles)
    ig = [n.info_gains for n in self.nodes]
    self.best_node_idx = ig.index(max(ig))

    return

  # ## NEED TO IMPLEMENT THIS FOR NEW CONCENPTION OF NODE (AS WELL AS ALL THE OTHER FUNCTIONS)
  # # creates branches using profiles by calling the ProfileGroup pickbranch(self, node) method for the ProfileGroup in index profile_group_idx in self.profile_groups
  # def makeBranchesP(self, profile_group_idx = None):
  #   if profile_group_idx and len(self.profiles) <= profile_group_idx:
  #     print(f'self.profiles does not have index {profile_group_idx}')
  #     return

  #   if profile_group_idx and len(self.profiles[profile_group_idx]) == 0:
  #     print(f'no profiles exists in index {profile_group_idx }')
  #     return

  #   if profile_group_idx:
  #     b_maker_ps = self.profiles[profile_group_idx]
  #   else:
  #     b_maker_ps = [p for p_group in self.profiles for p in p_group]

  #   self.b = multiThread(Profile.answerQ, [[p, self.q] for p in b_maker_ps])
  #   return

  # # aggregate branches that are the same, sets self.b to a list of strings GIVEN self.b is not none
  # def aggregateBranches(self):
  #   if not self.b:
  #     print('this node has no branches')
  #     return

  #   self.b = list(dict.fromkeys(self.b))
  #   prompt = f'You are taking an English test.\nHere is a question: "{self.q}"\nHere is a python list of possible answers to the question: {self.b}\nThe current python list may contain answers that are worded differently, but have the same meaning. Remove all the duplicate answers and return a python list of unique answers:'
  #   self.b = str2lst(generateText([{'role': 'user', 'content': prompt}]))
  #   return

# Conversartion Functions

In [12]:
def startConv(i_want_sen = 'I want to watch a movie'):
  prompt = stichPrompt(prompts['startConv'][0], [i_want_sen])
  # prompt = f"""You are taking an English test.\nHere is a sentence: "{i_want_sen}, but I do not know [insert word]. Help me by asking me a question at a time."\nReturn the sentence but with the correct word filled in the empty slot:"""
  prompt = generateText([{'role': 'user', 'content': prompt}]).strip(""" '" """)

  context = [{'role': 'user', 'content': prompt}]
  # appendContext(generateText([{'role': 'user', 'content': prompt}]), context, 'assistant')

  return context

def continueConvBool(chat_hist):
  assert chat_hist[-1]['role'] == 'assistant', 'chat history should have the assistant as the last message'
  prompt = stichPrompt(prompts['continueConvBool'][0], [chat_hist[-1]['content']])

  return 'question' in generateText([{'role': 'user', 'content': prompt}]).lower()

In [13]:
def convStep(context):
  return

#Main

In [27]:
prompts = {'startConv': ["""You are taking an English test.\nHere is a sentence: "{i_want_sen}, but I do not know [insert word]. Help me by asking me a question at a time."\nReturn the sentence but with the correct word filled in the empty slot:"""],

           'genRealProfile': ["Generate an example profile of a possible user. The profile should be a short sequence of keywords about their personal preferences regarding {domain}."],

           'continueConvBool': ["Is the following text a question or a suggestion?/n/n{text}/n/nAnswer [Question, Suggestion]:"],

           'genUProfiles': ['Generate {number} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding {domain}.'],

           'extractLastPreference': ['Here is a conversation.\nASSISTANT:\n{question}\nUSER:\n{answer}\nTASK: Summarize what we know about the user in a single sentence.\nSUMMARY:'],

           'genYProfiles': ['Here is a list of user preferences:{bullet_points}',
                            'Here is a sentence describing the preferences of a user:{sentence}\n',
                            'Create {number} different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:',
                            'Create different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'],

           'answerQ': ['You are a character. Here is what you know about your character: {profile}\nYou are speaking to an assistant and you speak in brief sentences.\nAnswer the assistant in character.\nAssistant: {question}\nYou:'],

           'pickBranch': ["""You are the following character: {profile}\nYou are asked the following question: {question}\n\nFrom the options below, which is most likely your answer to the question?\n{bullet_points}\nRemember, you have to pick one from the list or answer "None of the answers in the list". Only return the most likely answer, followed by the number that is the place of the answer in the list.\nAnswer, Number:""",
                          "You are the following character: {profile}\nYou are asked the following question: {question}\nIs it likely that you would answer: {answer}?\n\n[Yes, No]:"],

          'makeBranches': ['Here is a question:\n"{question}"\nReturn the smallest numbered list of different possible answers to this question. Make sure that this list, althought small, covers how any user could answer this question.\nList of answers:'],

           'makeNodes': ['Generate a python list of {node_n} questions you could possibly ask me at this point of the conversation.',
                         'Generate a python list of questions you could possibly ask me at this point of the conversation.',
                         ' Inlcude the last question you just asked in the python list.',
                         "Let's say I said the following: {answer}\n {prompt}"],


           }

In [15]:
# generate u profiles
u_profiles = genUProfiles(10, 'traveling')

In [28]:
# generate the real profile
real_profile = genRealProfile('traveling')

In [30]:
# printing real profile
pprint.pprint(real_profile.text)
# printing their response to the following question as a quick little test
pprint.pprint(real_profile.answerQ(real_profile, 'Are you more interested in exploration or relaxation?'))

('Adventure seeker, solo traveler, budget-conscious, off-the-beaten-path, '
 'nature lover, hiking enthusiast, cultural immersion, local experiences, '
 'photography, backpacking, adrenaline junkie, camping, road trips, foodie, '
 'sustainable travel, historical sites, beach lover, spontaneous, wanderlust, '
 'wildlife encounters, authentic souvenirs')
'Exploration.'


In [34]:
# initialize layer
layer = Layer(startConv('I want to travel'))

In [35]:
# creating nodes in layer, hence possible questions
layer.makeNodes(3)

In [36]:
[n.question for n in layer.nodes]

['Have you ever traveled before? If so, which destinations did you enjoy the most?',
 'What are your interests or hobbies? Are there any specific activities or experiences you would like to try during your travels?',
 'Do you have any budget constraints or preferences in terms of the cost of travel?']

In [37]:
# creating the branches for each of the nodes (without using profiles)
layer.makeBranches()

In [38]:
pprint.pprint([n.branches for n in layer.nodes])

[['Yes, I have traveled before. My favorite destinations were Paris, Tokyo, '
  'and New York.',
  'Yes, I have traveled before. The destinations I enjoyed the most were the '
  'beach destinations like Bali and Maldives.',
  'No, I have never traveled before.'],
 ['Reading, photography, hiking, and cooking.',
  'Playing sports, exploring different cultures, and trying out local cuisine.',
  'Painting, attending music festivals, and learning new languages.',
  'Yoga, scuba diving, and immersing in nature.',
  'Camping, fishing, and visiting historical landmarks.'],
 ['Yes, I have budget constraints and prefer low-cost travel.',
  'No, I dont have any budget constraints and am open to any cost of travel.',
  'Yes, I have budget constraints but Im willing to consider various price '
  'ranges for travel.',
  'Somewhat, I have certain preferences for cost but can be flexible depending '
  'on the travel options available.',
  'I have specific budget limitations and can only afford travel 

In [39]:
# layer.nodes[0].tst_b

In [41]:
# split the profiles
layer.profiles = u_profiles
layer.splitProfiles()

In [42]:
for n in layer.nodes: print(n.profile_branch_map)

[3, 0, 1, 0, 3, 0, 3, 3, 3, 3]
[6, 1, 3, 1, 3, 1, 3, 6, 3, 1]
[2, 2, 2, 3, 2, 2, 2, 2, 2, 2]


In [43]:
for n_idx in range(len(layer.nodes)):
  print(f'////////// {layer.nodes[n_idx].question} //////////\n')
  for p_idx in range(len(layer.profiles)):
    print(f'{layer.profiles[p_idx].text}')
    choice = layer.nodes[n_idx].profile_branch_map[p_idx]
    print(f'{layer.nodes[n_idx].branches[choice]}\n')
    # if choice is not None: print(f'{layer.nodes[n_idx].branches[choice]}\n')
    # else: print('None\n')

////////// Have you ever traveled before? If so, which destinations did you enjoy the most? //////////

Adventure Seeker: Always on the lookout for adrenaline-pumping activities like skydiving, bungee jumping, and rock climbing in exotic destinations.
NONE OF THE ABOVE

Culture Enthusiast: Fascinated by historical landmarks, museums, and local traditions, with a passion for exploring diverse cultures around the world.
Yes, I have traveled before. My favorite destinations were Paris, Tokyo, and New York.

Beach Lover: Craves sun-soaked shores, crystal-clear waters, and relaxation, always on the hunt for the perfect tropical paradise.
Yes, I have traveled before. The destinations I enjoyed the most were the beach destinations like Bali and Maldives.

Foodie Explorer: Driven by the desire to indulge in the finest cuisines, street food, and local delicacies, making culinary experiences a priority while traveling.
Yes, I have traveled before. My favorite destinations were Paris, Tokyo, and 

In [44]:
layer.calcInfoGain()

In [45]:
for n in layer.nodes:
  print(n.info_gains)

print(layer.nodes[layer.best_node_idx].question)

1.295461844238322
1.5219280948873621
0.4689955935892809
What are your interests or hobbies? Are there any specific activities or experiences you would like to try during your travels?


In [46]:
layer.context

[{'role': 'user',
  'content': 'I want to travel, but I do not know where. Help me by asking me a question at a time.'}]

# OLD CODE

In [14]:
def choose_with_idk(options, p_idk = 0.2):
  return random.choices([random.choice(options), "I don't know"], weights = [1-p_idk, p_idk], k=1)[0]

In [15]:
def choose_branch(q, branches, profiles):
  ans = []

  for p in profiles:
    prompt = f"""You are the following character: {p}\nYou are asked the following question: {q}\n\nFrom the options below, which is most likely your answer to the question?\n{list2numbered(branches)}\nRemember, you have to pick one from the list. Only return the most likely answer, followed by the number that is the place of the answer in the list.\nAnswer, Number:"""
    txt = generateText([{'role': 'user', 'content': prompt}])
    m = re.search('\d', txt)
    if m:
      s = m.start()
      e = m.end()
      ans.append(int(txt[s:e]))
    else: ans.append(0)


  return ans

In [13]:
def genAnswP(profile, q):
  prompt = f'You are a character. Here is what you know about your character: {profile}\nYou are speaking to an assistant and you speak in brief sentences.\nAnswer the assistant in character.\nAssistant: {q}\nYou:'
  return generateText([{'role': 'user', 'content': prompt}])

def genAns(q):
  prompt = f'Here is a question:\n"{q}"\nReturn a python list with all the possible answers to the question:'
  txt = generateText([{'role': 'user', 'content': prompt}], tries = 2)
  return str2lst(txt)

def aggregateAns(ans_lst, q = None):
  print('aggregating')
  ans_lst = list(dict.fromkeys(ans_lst))
  #prompt = f'Here is a list of preferences: {ans_lst}\nRemove all the answers that mean the same thing and return a list of unique answers:' BEST
  # prompt = f'Here is a list of preferences: {ans_lst}\nRemove all the answers that are too similar and return a small list of unique answers:'
  if q:
    #prompt = f'You will be given a python list that has possible answers to the question "{q}"\nYour task is to remove all the answers that mean the same thing and return a small list of unique answers.\nHere is the python list: {ans_lst}'
    prompt = f'You are taking an English test.\nHere is a question: "{q}"\nHere is a python list of possible answers to the question: {ans_lst}\nThe current python list contains answers that are worded differently, but contain the same meaning. Remove all the duplicate answers and return a python list of unique answers:'
  else:
    prompt = f'Here is a list of preferences: {ans_lst}\nRemove all the answers that mean the same thing and return a list of unique answers:'

  return str2lst(generateText([{'role': 'user', 'content': prompt}]))

In [ ]:
# Class representing a group of profiles
class ProfileGroup:
  def __init__(self, preference_lst = None):
    self.preference_lst = preference_lst
    self.profiles = []
    # if preference_lst:
    #   self.profiles = self.makeYProfiles()
    #   self.pType = ProfileType.Y
    # else:
    #   self.profiles = self.makeUProfiles()
    #   self.pType = ProfileType.U
    return

  # each profile answers question q, returns list of strings
  def answerQ(self, q):
    return multiThread(Profile.answerQ, [[p, self.q] for p in self.profiles])

  # each profile picks branch that is best fit, returns list of int
  def pickBranch(self, node):
    return multiThread(Profile.pickBranch, [[p, node] for p in self.profiles])

  def makeUProfiles(self, prof_num):
    prompt = f'Generate {prof_num} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding traveling.'
    txt = generateText([{'role': 'user', 'content': prompt}], tries = 2)
    self.profiles.extend(str2lst(txt))
    return

  def makeYProfiles(self):
    p_n = len(self.preference_lst)
    prompt = self.preference_lst[0]

    # if preference_lst has more than one preferece, create a single sentence representing all of them
    if p_n > 1:
      prompt = f'Here is a list of user preferences:{list2numbered(self.preference_lst)}'
      prompt = generateText([{'role': 'user', 'content': prompt+'Create a single sentence of key words that describes all of these preferences:'}], tries = 2)

    prompt = f'Here is a sentence describing the preferences of a user:{prompt}\n'

    # if profile number is specified, generate this specific number, otherwise let LLM decide
    prompt += f'Create {self.prof_num} different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'
    # prompt += 'Create different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'

    return str2lst(generateText([{'role': 'user', 'content': prompt}], tries = 2))

In [ ]:
qs_ans = [[] for q in range(len(p_qs))]
for idx, a in enumerate(ans): qs_ans[idx%len(p_qs)].append(a)
print(len(qs_ans))
print(len(qs_ans[0]))

In [ ]:
qs_ans[0]

In [ ]:
# aggregating answers
branches = [[] for q in range(len(p_qs))]
for i, ans in enumerate(qs_ans):
  branches[i] = aggregateAns(ans)

for b in branches:
  print(len(b))
pprint.pprint(branches)

In [ ]:
branches = [[] for q in range(len(p_qs))]
for i, ans in enumerate(qs_ans):
  branches[i] = aggregateAns(ans, p_qs[i])

for b in branches:
  print(len(b))
pprint.pprint(branches)